In [ ]:
# Importer les bibliothèques nécessaires
from moabb.datasets import BNCI2014_001
from pyriemann.estimation import Covariances
from pyriemann.classification import MDM
from sklearn.model_selection import cross_val_score

In [4]:
# Importer les bibliothèques nécessaires
from moabb.datasets import BNCI2014_001
from pyriemann.estimation import Covariances
from pyriemann.classification import MDM
from sklearn.model_selection import cross_val_score

# Charger le dataset BNCI2014_001
dataset = BNCI2014_001()

# Diviser le dataset en données d'entraînement et de test
X, y, metadata = dataset.get_data()

# Calculer les covariances des données
covariances = Covariances(estimator='lwf').fit_transform(X)

# Créer et évaluer le modèle MDM avec la validation croisée
clf = MDM()
scores = cross_val_score(clf, covariances, y, cv=5)

# Afficher les scores de validation croisée
print("Scores de validation croisée :", scores)
print("Précision moyenne :", scores.mean())
